In [1]:
# load packages
import pandas as pd
import numpy as np
from statsmodels.tools import eval_measures
from sklearn.model_selection import train_test_split

import statsmodels.api as sm

# turn off scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

from sklearn.metrics import mean_absolute_error as MAE
from sklearn.linear_model import LinearRegression

In [2]:
train_data = pd.read_csv ("train_2017.csv")
properties_data = pd.read_csv ("properties_2017.csv")

In [3]:
combined_data = pd.merge(train_data, properties_data, how = "inner", on = ["parcelid"])

In [4]:
formula = []
null_columns = combined_data.columns[combined_data.isnull().any()]
for col in combined_data.columns:
    try:
        if col in null_columns:
            combined_data[col].fillna((combined_data[col].mean()),inplace = True)
        if col != "logerror" and col != "transactiondate":
            formula.append(col)
    except:
        print("Not able to fill: " + col)

Not able to fill: propertycountylandusecode
Not able to fill: propertyzoningdesc
Not able to fill: taxdelinquencyflag


In [5]:
train, test = train_test_split(combined_data, test_size = 0.25, random_state = 42)

In [6]:
train.shape

(58209, 60)

In [7]:
test.shape

(19404, 60)

In [8]:
train.head(10)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
11042,12845746,-0.04106,2017-02-21,1.81201,7.38647,679.72000,4.00000,3.00000,3.93333,9.00000,...,1.43429,1.00000,239879.00000,334616.00000,2016.00000,94737.00000,4225.07000,NaN,14.08828,60375001001006.00000
38028,11014702,0.01144,2017-05-19,1.00000,7.38647,679.72000,2.00000,4.00000,3.93333,8.00000,...,1.43429,1.00000,258883.00000,615624.00000,2016.00000,356741.00000,7521.32000,NaN,14.08828,60371066411005.00000
76193,17210539,-0.02439,2017-09-13,1.81201,7.38647,679.72000,2.00000,4.00000,3.93333,6.53378,...,1.00000,1.00000,286479.00000,572958.00000,2016.00000,286479.00000,6480.12000,NaN,14.08828,61110063021008.00000
37498,12909665,0.00091,2017-05-18,1.81201,7.38647,679.72000,2.00000,2.00000,3.93333,6.00000,...,1.43429,1.00000,98745.00000,116532.00000,2016.00000,17787.00000,1696.07000,NaN,14.08828,60374038022002.00000
35580,11491800,-0.05815,2017-05-12,1.81201,7.38647,679.72000,3.00000,4.00000,3.93333,11.00000,...,1.43429,1.00000,390815.00000,971635.00000,2016.00000,580820.00000,11479.59000,NaN,14.08828,60376206013001.00000
66718,13099917,-0.01426,2017-08-15,1.00000,7.38647,679.72000,2.00000,3.00000,3.93333,8.00000,...,1.43429,1.00000,148637.00000,329493.00000,2016.00000,180856.00000,3986.76000,NaN,14.08828,60374033042000.00000
27638,11660583,-0.03963,2017-04-19,1.00000,7.38647,679.72000,7.00000,5.00000,3.93333,11.00000,...,1.43429,1.00000,1007264.00000,2494966.00000,2016.00000,1487702.00000,30189.06000,NaN,14.08828,60372623033003.00000
49495,17164537,-0.14860,2017-06-22,1.81201,7.38647,679.72000,2.00000,4.00000,3.93333,6.53378,...,1.00000,1.00000,50714.00000,65164.00000,2016.00000,14450.00000,735.88000,NaN,14.08828,61110037001017.00000
46859,12377044,-0.11874,2017-06-15,1.81201,7.38647,679.72000,1.00000,2.00000,3.93333,4.00000,...,1.43429,1.00000,21513.00000,47792.00000,2016.00000,26279.00000,1238.70000,NaN,14.08828,60375301013009.00000
43662,12826444,-0.01729,2017-06-06,1.81201,7.38647,679.72000,1.00000,3.00000,3.93333,4.00000,...,1.43429,1.00000,65827.00000,133148.00000,2016.00000,67321.00000,1864.79000,NaN,14.08828,60375035021005.00000


In [9]:
train_X = train[formula]

In [10]:
multi_lm = LinearRegression(n_jobs=-1).fit(train_X, train.logerror)

In [11]:
test_X = test[formula]

In [12]:
# Get prediction
pred = multi_lm.predict(test_X)

In [13]:
100-(eval_measures.rmse(test["logerror"], pred)*100)

81.66592588635177

In [14]:
100-(MAE(pred,test["logerror"])*100)

92.75029630902645

In [15]:
combined_data.corr()

,parcelid,logerror,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
parcelid,1.00000,0.01541,0.09713,0.00021,0.00025,0.00323,0.00681,-0.00040,-0.03431,0.01455,...,0.00011,0.13378,0.01879,0.02917,0.01250,nan,0.00401,0.00222,-0.00237,0.06192
logerror,0.01541,1.00000,0.00431,-0.00034,0.00999,0.02579,0.03160,0.00878,-0.00982,0.02879,...,-0.00069,0.00483,0.00414,0.00840,0.00342,nan,0.00050,0.00220,-0.01936,0.00455
airconditioningtypeid,0.09713,0.00431,1.00000,0.00172,-0.00000,-0.02233,0.04979,-0.00000,-0.09628,-0.02204,...,-0.00000,-0.03418,0.09690,-0.04103,-0.00845,nan,0.00791,-0.02405,0.00068,0.01938
architecturalstyletypeid,0.00021,-0.00034,0.00172,1.00000,-0.00000,-0.00014,-0.00011,-0.00000,-0.00000,-0.00014,...,-0.00000,-0.00146,-0.00363,-0.00081,0.00005,nan,0.00044,-0.00017,0.00012,0.00000
basementsqft,0.00025,0.00999,-0.00000,-0.00000,1.00000,0.02391,0.01213,0.00000,0.00000,0.02442,...,-0.02811,0.01121,0.00524,0.00556,0.00883,nan,0.00912,0.00821,0.00000,0.00000
bathroomcnt,0.00323,0.02579,-0.02233,-0.00014,0.02391,1.00000,0.63548,0.00000,0.43053,0.97899,...,0.00144,0.37024,0.12749,0.56492,0.46101,nan,0.34743,0.46596,0.00751,0.00119
bedroomcnt,0.00681,0.03160,0.04979,-0.00011,0.01213,0.63548,1.00000,-0.00000,0.08758,0.60120,...,-0.00354,0.04734,0.06785,0.29588,0.23932,nan,0.17895,0.24636,0.00277,0.00780
buildingclasstypeid,-0.00040,0.00878,-0.00000,-0.00000,0.00000,0.00000,-0.00000,1.00000,-0.00286,-0.00000,...,0.00000,0.00226,0.00191,-0.01961,-0.00775,nan,-0.00112,-0.01094,0.00156,0.00006
buildingqualitytypeid,-0.03431,-0.00982,-0.09628,-0.00000,0.00000,0.43053,0.08758,-0.00286,1.00000,0.43708,...,0.00000,0.47629,0.00022,0.31770,0.25228,nan,0.18609,0.25566,0.01097,-0.00128
calculatedbathnbr,0.01455,0.02879,-0.02204,-0.00014,0.02442,0.97899,0.60120,-0.00000,0.43708,1.00000,...,0.00147,0.37261,0.14067,0.58341,0.47798,nan,0.36164,0.48306,0.00737,0.00287
